# Data Wrangling + EDA

## Introduction

The goal of this notebook is to take the data on which Leandojo's ReProver was trained and to modify it so that we can train a model to output tactic suggestions using conjectures instead of just names of existing theorems. To do so, we go through each tactic in the data set and create a new entry where the named theorems are replaced by their content. For example, the tactic

    rw [Acc.ndrec, rec_eq_recC, Acc.ndrecC]

is expanded to

    rw [
    
    < c>.{u1, u2} {α : Sort u2} {r : α → α → Prop} {C : α → Sort u1}\n    (m : (x : α) → ((y : α) → r y x → Acc r y) → ((y : α) → (a : r y x) → C y) → C x)\n    {a : α} (n : Acc r a) : C a< /c>,

    < c> : @Acc.rec = @Acc.recC< /c>,

    < c> {C : α → Sort v}\n    (m : (x : α) → ((y : α) → r y x → Acc r y) → ((y : α) → (a : r y x) → C y) → C x)\n    {a : α} (n : Acc r a) : C a< /c>
    
    ]

by using the content for each existing theorem stored in the corpus.

The notebook is to be run inside of the Leandojo benchmark folders (available at https://zenodo.org/records/10929138 ), either novel_premises or random.

Of the tens of thousands of theorems used in proofs, a few thousand raise errors when retrieving their original content, so we run the notebook once to catch those theorem names as exceptions, saving it as a json. We restart the notebook, loading the exceptions json, leaving the names of exceptions but replacing with content for all other theorems.

## Import packages and load data

In [1]:
import json
import re

In [2]:
with open("train.json", "r") as read_file:
    data = json.load(read_file)

In [3]:
data[1]

{'url': 'https://github.com/leanprover-community/mathlib4',
 'commit': 'fe4454af900584467d21f4fd4fe951d29d9332a7',
 'file_path': '.lake/packages/std/Std/WF.lean',
 'full_name': 'Acc.ndrec_eq_ndrecC',
 'start': [76, 18],
 'end': [78, 42],
 'traced_tactics': [{'tactic': 'funext α r motive intro a t',
   'annotated_tactic': ['funext α r motive intro a t', []],
   'state_before': '⊢ @ndrec = @Acc.ndrecC',
   'state_after': 'case h.h.h.h.h.h\nα : Sort u_1\nr : α → α → Prop\nmotive : α → Sort u_2\nintro : (x : α) → (∀ (y : α), r y x → Acc r y) → ((y : α) → r y x → motive y) → motive x\na : α\nt : Acc r a\n⊢ ndrec intro t = Acc.ndrecC intro t'},
  {'tactic': 'rw [Acc.ndrec, rec_eq_recC, Acc.ndrecC]',
   'annotated_tactic': ['rw [<a>Acc.ndrec</a>, <a>rec_eq_recC</a>, <a>Acc.ndrecC</a>]',
    [{'full_name': 'Acc.ndrec',
      'def_path': '.lake/packages/lean4/src/lean/Init/WF.lean',
      'def_pos': [15, 22],
      'def_end_pos': [15, 31]},
     {'full_name': '_private.«.lake».packages.std.Std.

Each element of the data list corresponds to a theorem and its proof, stored as a dictionary. The tactics (i.e., lines of the proof) and the information about each step of the proof are stored as a list under 'traced_tactics'. Our script will go through each 'annotated_tactic' and replace the names of the theorems (which show up between < a> and < \a>) with the content of those theorems (unless the theorem name is among the exceptions), and store the resulting string under 'expanded_tactic'.

In [4]:
with open("..\\corpus.jsonl", "r") as read_file:
    corpus = [json.loads(line) for line in read_file]

In [5]:
corpus[1]

{'path': '.lake/packages/lean4/src/lean/Init/Coe.lean',
 'imports': ['.lake/packages/lean4/src/lean/Init/Prelude.lean'],
 'premises': [{'full_name': 'Coe',
   'code': 'class Coe (α : semiOutParam (Sort u)) (β : Sort v) where\n  \n  coe : α → β',
   'start': [120, 1],
   'end': [129, 14],
   'kind': 'commanddeclaration'},
  {'full_name': 'CoeTC',
   'code': 'class CoeTC (α : Sort u) (β : Sort v) where\n  \n  coe : α → β',
   'start': [132, 1],
   'end': [139, 14],
   'kind': 'commanddeclaration'},
  {'full_name': 'CoeOut',
   'code': 'class CoeOut (α : Sort u) (β : semiOutParam (Sort v)) where\n  \n  coe : α → β',
   'start': [146, 1],
   'end': [152, 14],
   'kind': 'commanddeclaration'},
  {'full_name': 'CoeOTC',
   'code': 'class CoeOTC (α : Sort u) (β : Sort v) where\n  \n  coe : α → β',
   'start': [155, 1],
   'end': [162, 14],
   'kind': 'commanddeclaration'},
  {'full_name': 'CoeHead',
   'code': 'class CoeHead (α : Sort u) (β : semiOutParam (Sort v)) where\n  \n  coe : α → β',


Each entry in the corpus is the data of a particular Lean file, with a list of the data from theorems contained in it. To replace a theorem name with its content, we search the corpus by using the Lean file and theorem name, accessing the string stored as 'code' in the corresponding dictionary.

On first run-through, adjust the commenting in the following cell.

In [6]:
# COMMENT OUT ON FIRST RUN-THROUGH
with open("exceptions.json", "r") as read_file:
    exceptions = json.load(read_file)

# # UN-COMMENT ON FIRST RUN-THROUGH
# exceptions = []

In [7]:
exceptions[:10]

['eqToHom_trans',
 'inv_eq_of_hom_inv_id',
 'inv_eq_of_hom_inv_id',
 'inv_eq_of_hom_inv_id',
 'inv_eq_of_hom_inv_id',
 'inv_eq_of_hom_inv_id',
 'eq_inv_of_hom_inv_id',
 'IsIso.eq_inv_of_hom_inv_id',
 'pullbackSymmetry_hom_comp_fst',
 'IsIso.eq_inv_of_hom_inv_id']

In [8]:
# COMMENT OUT ON FIRST RUN-THROUGH
exceptions = set(exceptions)

In [9]:
len(exceptions)

4880

Although there are thousands of exceptions, there are many more theorems which are replaced successfully. Many of the exceptions are relatively simple or low-level theorems, so it makes sense for the model to memorize the names of them instead of their content.

## Define Functions

In [10]:
def find_premise_in_lean_file(premise_list, start, even_shorter):
    """Return theorem data for the premise with position start and name even_shorter."""
    max_start = len(premise_list)
    left, right = 0, max_start
    while left < right:     # binary search
        mid = left + (right - left) // 2
        mid_start = premise_list[mid]['start'][0]
        mid_name = premise_list[mid]['full_name']
        error_bound = 20 + (start // 10) # for some reason, the starting point is shifted in the corpus vs the train json, so we allow some wiggle room.
        if mid_start <= start and mid_start > start - error_bound and (mid_name.endswith(even_shorter) or mid_name.endswith(even_shorter + '\'')):
            return premise_list[mid]
        elif mid_start > start:
            right = mid
        else:
            left = mid + 1
    raise ValueError

In [11]:
def get_code(corpus, path, start, even_shorter):
    """Return the code for the theorem with position start and name even_shorter."""
    for file_dict in corpus:
        if file_dict['path'] == path:
            premise = find_premise_in_lean_file(file_dict['premises'], start, even_shorter)
            return premise['code']
    raise ValueError

In [12]:
def content_from_code(code, even_shorter):
    """Return the content of a theorem from its code and its short name."""
    new_code = code.split(' :=')[0] # the code 
    new_code = new_code.split(even_shorter)[1]
    return new_code

In [13]:
# combining the above functions to retrieve a theorem's content.
def get_content(short_name, def_path, def_pos):
    """Return the content of a theorem from its short name, file path, and position."""
    if short_name.endswith('\''):
        short_name = short_name[:-1]
    even_shorter = short_name.split('.')[-1]    # we want to make the name as short as possible so as not to miss potential matches.
    code = get_code(corpus, def_path, def_pos[0], even_shorter)
    content = content_from_code(code, even_shorter)
    return content

In [14]:
def replace_single_name_with_content(annotated_tactic_str, short_name, def_path, def_pos):
    """Return annotated_tactic_str with the theorem name replaced with its content."""
    content = get_content(short_name, def_path, def_pos)
    return annotated_tactic_str.replace('<a>' + short_name + '</a>', '<c>' + content + '</c>')

In [15]:
def remove_annotation_from_name(annotated_tactic_str, short_name):
    """Return annotated_tactic_str with <a> and </a> removed from around short_name."""
    return annotated_tactic_str.replace('<a>' + short_name + '</a>', short_name)

In [16]:
def tactic_with_content(annotated_tactic):
    """Return the tactic with theorem names replaced with content."""
    global new_val_exceptions   # when first running through, we store all exceptions to save as a json which we use on re-running.
    global new_ind_exceptions
    annotated_tactic_str = annotated_tactic[0]
    short_names = [m[0] for m in list(re.finditer(r"<a>(?P<ident>.+?)</a>", annotated_tactic_str))]
    for k, sn in enumerate(short_names):
        shorter = sn[3:-4]
        l = len(shorter)
        full_name = annotated_tactic[1][k]['full_name']
        if full_name[-l:] == shorter and shorter not in exceptions:
            def_path = annotated_tactic[1][k]['def_path']
            def_pos = annotated_tactic[1][k]['def_pos']
            try:
                annotated_tactic_str = replace_single_name_with_content(annotated_tactic_str, shorter, def_path, def_pos)
            except ValueError:
                new_val_exceptions = new_val_exceptions + [shorter]
            except IndexError:
                new_ind_exceptions = new_ind_exceptions + [shorter]
        else:
            annotated_tactic_str = remove_annotation_from_name(annotated_tactic_str, shorter)
    return annotated_tactic_str

## Run Code

On first run-through, the following code only serves to collect all of the exceptions. Many of the resulting 'expanded_tactic' entries will not be correctly updated and will still contain theorem names between < a> ... < /a>. Only upon second run-through, with the exceptions json loaded correctly, will the resulting 'expanded_tactic' entries be what we are looking for.

In [17]:
new_val_exceptions = []
new_ind_exceptions = []

for i, thm in enumerate(data):
    for tactic in thm['traced_tactics']:
        tactic['expanded_tactic'] = tactic_with_content(tactic['annotated_tactic'])

In [18]:
len(set(new_ind_exceptions)), len(set(new_val_exceptions))

(0, 0)

Upon second run-through, there should be zero new exceptions of either type.

In [19]:
new_exceptions = new_ind_exceptions + new_val_exceptions

## Save Data

In [ ]:
# # UNCOMMENT ON FIRST RUN THROUGH TO SAVE THE EXCEPTIONS JSON
# with open("exceptions.json", "w") as fp:
#         json.dump(new_exceptions, fp)

In [20]:
data[3]

{'url': 'https://github.com/leanprover-community/mathlib4',
 'commit': 'fe4454af900584467d21f4fd4fe951d29d9332a7',
 'file_path': '.lake/packages/std/Std/WF.lean',
 'full_name': 'WellFounded.fixF_eq_fixFC',
 'start': [113, 18],
 'end': [115, 36],
 'traced_tactics': [{'tactic': 'funext α r C F x a',
   'annotated_tactic': ['funext α r C F x a', []],
   'state_before': '⊢ @fixF = @WellFounded.fixFC',
   'state_after': 'case h.h.h.h.h.h\nα : Sort u_1\nr : α → α → Prop\nC : α → Sort u_2\nF : (x : α) → ((y : α) → r y x → C y) → C x\nx : α\na : Acc r x\n⊢ fixF F x a = WellFounded.fixFC F x a',
   'expanded_tactic': 'funext α r C F x a'},
  {'tactic': 'rw [fixF, Acc.rec_eq_recC, fixFC]',
   'annotated_tactic': ['rw [<a>fixF</a>, <a>Acc.rec_eq_recC</a>, <a>fixFC</a>]',
    [{'full_name': 'WellFounded.fixF',
      'def_path': '.lake/packages/lean4/src/lean/Init/WF.lean',
      'def_pos': [58, 19],
      'def_end_pos': [58, 23]},
     {'full_name': '_private.«.lake».packages.std.Std.WF.0.Acc.rec_

Now that each tactic has had an 'expanded_tactic' entry added to it in our data, we save the result as a new json.

In [ ]:
# COMMENT OUT ON FIRST RUN-THROUGH
with open("mytrain.json", "w") as fp:
        json.dump(data, fp)